# Demo: sample crawl

In [1]:
import pandas as pd
from selenium import webdriver
import time
from selenium.webdriver.chrome.service import Service
import unidecode
from urllib.parse import unquote

PATH = Service("C:\Program Files (x86)\chromedriver.exe")

driver = webdriver.Chrome(service = PATH)
driver.close()

In [2]:
whoscored_url=[
    "https://1xbet.whoscored.com/Regions/252/Tournaments/2/England-Premier-League",
    "https://1xbet.whoscored.com/Regions/108/Tournaments/5/Italy-Serie-A",
    "https://1xbet.whoscored.com/Regions/206/Tournaments/4/Spain-LaLiga",
    "https://1xbet.whoscored.com/Regions/81/Tournaments/3/Germany-Bundesliga",
    "https://1xbet.whoscored.com/Regions/74/Tournaments/22/France-Ligue-1"
]

In [3]:
PATH = Service("C:\Program Files (x86)\chromedriver.exe")

driver = webdriver.Chrome(service = PATH)
driver.get(whoscored_url[0])

teams_url_raw = \
driver.find_elements("xpath", "(//table[@class='grid with-centered-columns hover']/tbody/tr/td/a[@class='team-link '])")
teams_url = []

for team_url_raw in teams_url_raw:
    teams_url.append(team_url_raw.get_attribute('href'))
driver.close()
teams_url

['https://1xbet.whoscored.com/Teams/167/Show/England-Manchester-City',
 'https://1xbet.whoscored.com/Teams/26/Show/England-Liverpool',
 'https://1xbet.whoscored.com/Teams/15/Show/England-Chelsea',
 'https://1xbet.whoscored.com/Teams/30/Show/England-Tottenham',
 'https://1xbet.whoscored.com/Teams/13/Show/England-Arsenal',
 'https://1xbet.whoscored.com/Teams/32/Show/England-Manchester-United',
 'https://1xbet.whoscored.com/Teams/29/Show/England-West-Ham',
 'https://1xbet.whoscored.com/Teams/14/Show/England-Leicester',
 'https://1xbet.whoscored.com/Teams/211/Show/England-Brighton',
 'https://1xbet.whoscored.com/Teams/161/Show/England-Wolverhampton',
 'https://1xbet.whoscored.com/Teams/23/Show/England-Newcastle',
 'https://1xbet.whoscored.com/Teams/162/Show/England-Crystal-Palace',
 'https://1xbet.whoscored.com/Teams/189/Show/England-Brentford',
 'https://1xbet.whoscored.com/Teams/24/Show/England-Aston-Villa',
 'https://1xbet.whoscored.com/Teams/18/Show/England-Southampton',
 'https://1xbe

In [4]:
PATH = Service("C:\Program Files (x86)\chromedriver.exe")

driver = webdriver.Chrome(service = PATH)
driver.get(teams_url[1])

players_url_raw = driver.find_elements("xpath","//tbody[@id='player-table-statistics-body']/tr/td/a[@class='player-link']")
players_url = []

for player_url_raw in players_url_raw:
    players_url.append(unidecode.unidecode(unquote(player_url_raw.get_attribute('href').lower().replace('show','history'))))
driver.close()
players_url

KeyboardInterrupt: 

## tabular extract

In [ ]:
linkz = players_url[0]
PATH = Service("C:\Program Files (x86)\chromedriver.exe")

driver = webdriver.Chrome(service = PATH)
driver.get(linkz)

#table structure
cols = driver.find_elements('xpath',"//table[@id='top-player-stats-summary-grid']/thead/tr/th")[2:]

for i in range(len(cols)):
    cols[i] = cols[i].get_attribute('innerHTML')
    

data =  driver.find_elements('xpath',"//tbody[@id='player-table-statistics-body']/tr")

pd_appender = []
for dt in data[:-1]:
    dtnr = {'name':linkz[linkz.find('history/')+8:]}
    dtnr.update(zip(cols,dt.text.split()))
    pd_appender.append(dtnr)

driver.close()


In [ ]:
e_table = pd.DataFrame(pd_appender)
e_table

## apply data type for columns & grouping

In [ ]:
e_table['Season'] = e_table['Season'].apply(lambda ss:ss[:4])
e_table['Apps'] = e_table['Apps'].apply\
(lambda apps: int(apps[:apps.find('(')])+int(apps[apps.find('(')+1:apps.find(')')]) if not apps.isdigit() else int(apps))
e_table['Mins'] = e_table['Mins'] .apply(lambda s: int(s) if s.isdigit() else 0)
e_table['Goals'] = e_table['Goals'] .apply(lambda s: int(s) if s.isdigit() else 0)
e_table['Assists'] = e_table['Assists'] .apply(lambda s: int(s) if s.isdigit() else 0)
e_table['Yel'] = e_table['Yel'] .apply(lambda s: int(s) if s.isdigit() else 0)
e_table['MotM'] = e_table['MotM'] .apply(lambda s: int(s) if s.isdigit() else 0)
e_table['Red'] = e_table['Red'] .apply(lambda s: int(s) if s.isdigit() else 0)
e_table['Rating'] = e_table['Rating'] .apply(lambda s: float(s) if s != '-' else 0)

In [ ]:
e_table

In [ ]:
e_table = e_table.iloc[:,[0,1,4,5,6,7,8,9,-2,-1]]
e_table

In [ ]:
def func_agg(x):
    d = []
    for col in (list(e_table)[2:-1]):
        d.append(x[col].sum())
    d.append((x['Rating']*x['Apps']).sum()/(x['Apps'].sum()))
    return pd.Series(d, index = list(e_table)[2:])

In [ ]:
test = e_table.groupby(["name","Season"],as_index = False).apply(func_agg)
for col in list(test)[2:-1]:
    test[col] = test[col].astype('int16')
    print (col, test[col].dtype)
test['Rating'] = test['Rating'].apply(lambda rt: float("%.2f" %rt))
test

# Final crawl

In [7]:
whoscored_url=[
    "https://1xbet.whoscored.com/Regions/252/Tournaments/2/England-Premier-League",
    "https://1xbet.whoscored.com/Regions/108/Tournaments/5/Italy-Serie-A",
    "https://1xbet.whoscored.com/Regions/206/Tournaments/4/Spain-LaLiga",
    "https://1xbet.whoscored.com/Regions/81/Tournaments/3/Germany-Bundesliga",
    "https://1xbet.whoscored.com/Regions/74/Tournaments/22/France-Ligue-1"
]

In [8]:
PATH = Service("C:\Program Files (x86)\chromedriver.exe")

driver = webdriver.Chrome(service = PATH)

teams_url = []

for url in whoscored_url:
    driver.get(url)

    teams_url_raw =\
    driver.find_elements("xpath", "(//table[@class='grid with-centered-columns hover']/tbody/tr/td/a[@class='team-link '])")


    for team_url_raw in teams_url_raw:
        teams_url.append(team_url_raw.get_attribute('href'))
driver.close()
len(teams_url)

98

In [11]:
PATH = Service("C:\Program Files (x86)\chromedriver.exe")

driver = webdriver.Chrome(service = PATH)
players_url = []
for team_url in teams_url:
    driver.get(team_url)
    #time.sleep(3)
    players_url_raw = driver.find_elements("xpath","//tbody[@id='player-table-statistics-body']/tr/td/a[@class='player-link']")

    for player_url_raw in players_url_raw:
        #rlink = 
        #if rlink not in players_url:
        players_url.append(unidecode.unidecode(unquote(player_url_raw.get_attribute('href').lower().replace('show','history'))))

driver.close()
len(players_url)


2926

In [15]:
len(set(players_url))

2795

In [12]:
f = open("player_url.txt","w")
for url in players_url:
    f.write(url+"\n")
f.close()
print(sum(1 for line in open('player_url.txt')))

2926


In [18]:
f = open("player_url.txt","r")
players_url = f.read().split()
f.close()
players_url

['https://1xbet.whoscored.com/players/73084/history/kevin-de-bruyne',
 'https://1xbet.whoscored.com/players/303139/history/rodri',
 'https://1xbet.whoscored.com/players/128967/history/joao-cancelo',
 'https://1xbet.whoscored.com/players/331254/history/phil-foden',
 'https://1xbet.whoscored.com/players/136741/history/bernardo-silva',
 'https://1xbet.whoscored.com/players/77464/history/ilkay-gundogan',
 'https://1xbet.whoscored.com/players/349760/history/ferran-torres',
 'https://1xbet.whoscored.com/players/279379/history/gabriel-jesus',
 'https://1xbet.whoscored.com/players/97692/history/raheem-sterling',
 'https://1xbet.whoscored.com/players/113069/history/jack-grealish',
 'https://1xbet.whoscored.com/players/145473/history/zack-steffen',
 'https://1xbet.whoscored.com/players/101374/history/john-stones',
 'https://1xbet.whoscored.com/players/273257/history/oleksandr-zinchenko',
 'https://1xbet.whoscored.com/players/104749/history/riyad-mahrez',
 'https://1xbet.whoscored.com/players/313

In [20]:
PATH = Service("C:\Program Files (x86)\chromedriver.exe")

driver = webdriver.Chrome(service = PATH)
pd_appender = []
#table structure
linkz = players_url[0]
driver.get(linkz)


cols = driver.find_elements('xpath',"//table[@id='top-player-stats-summary-grid']/thead/tr/th")[2:]

for i in range(len(cols)):
    cols[i] = cols[i].get_attribute('innerHTML')
driver.close()
cols

['Season',
 'Team',
 'Tournament',
 'Apps',
 'Mins',
 'Goals',
 'Assists',
 'Yel',
 'Red',
 'SpG',
 'PS%',
 'AerialsWon',
 'MotM',
 'Rating']

In [41]:
pd_appender = []

In [52]:
#crawl
PATH = Service("C:\Program Files (x86)\chromedriver.exe")

driver = webdriver.Chrome(service = PATH)

for linkz in players_url[:50]:
    driver.get(linkz)
    
    data =  driver.find_elements('xpath',"//tbody[@id='player-table-statistics-body']/tr")

    for dt in data[:-1]:
        dtnr = {'name':linkz[linkz.find('history/')+8:]}
        apd = []
        a = dt.text.split('\n')
        apd.append(a[0][:dt.text.find(' ')])
        apd.append(a[0][dt.text.find(' ')+1:dt.text.find('\n')])
        apd += a[1].split()
        dtnr.update(zip(cols, apd))
        pd_appender.append(dtnr)
driver.close()

In [53]:
e_table = pd.DataFrame(pd_appender)
e_table

,name,Season,Team,Tournament,Apps,Mins,Goals,Assists,Yel,Red,SpG,PS%,AerialsWon,MotM,Rating
0,kevin-de-bruyne,2021/2022,Belgium,WCQ,4,353,2,2,-,-,2.5,82.2,1.5,2,8.12
1,kevin-de-bruyne,2022/2023,Belgium,UNL,3,237,1,-,-,-,3,85.5,-,-,7.28
2,kevin-de-bruyne,2021/2022,Man City,EFLC,2,173,1,-,-,-,4,82.9,0.5,-,7.98
3,kevin-de-bruyne,2021/2022,Man City,UCL,9(1),735,2,3,2,-,2.4,85.5,0.2,1,7.15
4,kevin-de-bruyne,2021/2022,Man City,EPL,25(5),2205,15,8,2,-,2.6,82.5,0.4,4,7.64
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3428,joe-gomez,2017/2018,Liverpool,UCL,4(1),371,-,-,-,1,-,89,1.4,-,6.87
3429,joe-gomez,2017/2018,Liverpool,EPL,21(2),1901,-,2,3,-,0.2,84.5,1.4,-,7.11
3430,joe-gomez,2015/2016,Liverpool,UEL,2,180,-,-,-,-,1,90.7,2.5,-,7.30
3431,joe-gomez,2015/2016,Liverpool,EPL,5,438,-,1,3,-,0.2,78.8,1.8,-,6.86


In [54]:
e_table.drop_duplicates(keep = 'first', inplace = True)
e_table

,name,Season,Team,Tournament,Apps,Mins,Goals,Assists,Yel,Red,SpG,PS%,AerialsWon,MotM,Rating
0,kevin-de-bruyne,2021/2022,Belgium,WCQ,4,353,2,2,-,-,2.5,82.2,1.5,2,8.12
1,kevin-de-bruyne,2022/2023,Belgium,UNL,3,237,1,-,-,-,3,85.5,-,-,7.28
2,kevin-de-bruyne,2021/2022,Man City,EFLC,2,173,1,-,-,-,4,82.9,0.5,-,7.98
3,kevin-de-bruyne,2021/2022,Man City,UCL,9(1),735,2,3,2,-,2.4,85.5,0.2,1,7.15
4,kevin-de-bruyne,2021/2022,Man City,EPL,25(5),2205,15,8,2,-,2.6,82.5,0.4,4,7.64
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1371,luis-diaz,2020/2021,Porto,PLN,16(14),1638,6,5,-,-,1.4,83.4,0.8,4,6.96
1372,luis-diaz,2020/2021,Porto,UCL,4(5),451,2,-,2,-,0.6,80,0.4,-,6.63
1373,luis-diaz,2019/2020,Porto,PLN,20(9),1772,6,2,2,-,1.7,86.1,0.5,1,6.95
1374,luis-diaz,2019/2020,Porto,UEL,5(3),403,3,1,1,-,1,77.8,0.5,-,6.70
